
# Avance 2 — **Ingeniería de características**
**Proyecto:** Predictor de Escalamiento para Chatbot  
**Archivo base:** `generated_converted.csv`

Este proyecto se centra en desarrollar un modelo de aprendizaje automático que funcione como un predictor de escalamiento para un sistema de chatbot. Su finalidad es analizar cada turno de una conversación en tiempo real para decidir la acción más eficiente: continuar (continue) con el flujo automatizado, pedir una aclaración al usuario (clarify) o escalar la interacción a un agente humano (handoff), mejorando así la experiencia del usuario y la eficiencia del sistema.

El set de datos utilizado, generated_converted.csv, es un dataset sintético, lo que significa que fue generado artificialmente de forma programática para este proyecto. Se diseñó específicamente para simular de manera realista una amplia variedad de escenarios conversacionales, incluyendo diferentes intenciones, errores y niveles de ambigüedad.

## Nuestro Proceso de Feature Engineering

Para preparar los datos, aplicamos varias técnicas clave para que nuestro modelo aprenda mejor y más rápido.

---

### 1. Creación de Features

Enriquecimos los datos creando **nuevas variables** que capturan interacciones y patrones ocultos. Transformamos el texto en vectores numéricos usando **TF-IDF** y convertimos las variables categóricas con **one-hot encoding** para que el modelo pueda procesarlas.

---

### 2. Normalización y Escalado

Ajustamos las escalas de las variables para que ninguna dominara a las demás. Corregimos las distribuciones sesgadas con una transformación logarítmica (**`log1p`**) y luego estandarizamos todas las variables numéricas con **`StandardScaler`**. Esto ayuda a que el modelo converja más rápido.

---

### 3. Selección y Reducción

Para simplificar el modelo y evitar el ruido, eliminamos las características con baja varianza (**`VarianceThreshold`**) y las que estaban muy correlacionadas entre sí. Después, medimos la importancia de cada variable usando **chi-cuadrado (χ²)** para el texto y **ANOVA** para los datos numéricos. Finalmente, compactamos la información del texto en menos dimensiones con **TruncatedSVD**.

## 0) Carga de datos

In [1]:

import pandas as pd, numpy as np
CSV = r"generated_converted.csv"
df = pd.read_csv(CSV)
df['final_label'] = df['final_label'].astype(str).str.strip()
df['user_text'] = df['user_text'].fillna("")
print("Shape:", df.shape)
df.head(3)


Shape: (161, 20)


,conv_id,turn_id,user_text,bot_text,lang,domain,tool_available,tool_error,tool_timeout,n_reprompts_similar,confidence_sim,intent_margin_sim,has_dates,has_money,has_url,imperative_flag,wh_flag,topic_risky,final_label,timestamp
0,c_001,1,Quiero reservar un vuelo a Cancún para la próx...,"Claro, ¿para cuántos pasajeros y qué fechas ex...",es,reservas,1,0,0,0,0.72,0.31,1,0,0,1,0,0,clarify,2025-10-02T18:00:00Z
1,c_001,2,"para 2 adultos, del 10 al 15 de oct","Perfecto, buscando vuelos para 2 adultos a Can...",es,reservas,1,0,0,0,0.98,0.75,1,0,0,0,0,0,continue,2025-10-02T18:00:45Z
2,c_002,1,How much is the total for my cart? Include taxes.,"Your cart total is $1,450.50 USD, including a ...",en,calculo,1,0,0,0,0.95,0.60,0,0,0,0,1,0,continue,2025-10-02T18:01:10Z


## 0.1) Variables y utilidades

In [2]:

text_col = 'user_text'  # Columna de texto fuente para features lingüísticas.
dense_cols_raw = [
    # Features numéricas 
    'n_reprompts_similar','tool_error','tool_timeout',
    'confidence_sim','intent_margin_sim',
    'has_dates','has_money','has_url','imperative_flag','wh_flag','topic_risky','tool_available'
]
monitor_cats = ['lang','domain'] # Categorías usadas para análisis de sesgo (monitoreo).

# Asegurar que todas las columnas base existan en el DataFrame antes de usarlas.
for c in dense_cols_raw:
    if c not in df.columns: df[c] = 0
for c in monitor_cats:
    if c not in df.columns: df[c] = "N/A"

import pandas as pd, numpy as np, re
def build_dense_block(frame: pd.DataFrame) -> pd.DataFrame:
    """
    Genera un data frame con datos mas limpios
    """
    s = frame[text_col].fillna('')
    words = s.str.split()
    D = pd.DataFrame({
        'len_chars_user': s.str.len(),
        'len_words_user': words.apply(len),
        'question_marks': s.str.count(r'\?'),
        'exclam_marks': s.str.count(r'!'),
        'n_reprompts_similar': frame['n_reprompts_similar'].fillna(0).astype(float),
        'tool_error': frame['tool_error'].fillna(0).astype(float),
        'tool_timeout': frame['tool_timeout'].fillna(0).astype(float),
        'tool_error_or_timeout': (frame['tool_error'].fillna(0).astype(int) | frame['tool_timeout'].fillna(0).astype(int)).astype(int),
        'confidence_sim': frame['confidence_sim'].fillna(0.0).astype(float),
        'intent_margin_sim': frame['intent_margin_sim'].fillna(0.0).astype(float),
        'conf_to_margin_ratio': frame['confidence_sim'].fillna(0.0).astype(float) / (frame['intent_margin_sim'].fillna(0.0).astype(float) + 1e-6),
        'has_dates': frame['has_dates'].fillna(0).astype(int),
        'has_money': frame['has_money'].fillna(0).astype(int),
        'risk_or_money': ((frame['topic_risky'].fillna(0).astype(int) | frame['has_money'].fillna(0).astype(int))).astype(int),
        'has_url': frame['has_url'].fillna(0).astype(int),
        'imperative_flag': frame['imperative_flag'].fillna(0).astype(int),
        'wh_flag': frame['wh_flag'].fillna(0).astype(int),
        'topic_risky': frame['topic_risky'].fillna(0).astype(int),
        'tool_available': frame['tool_available'].fillna(0).astype(int),
    })
    # Binning para variables continuas
    for col, q in [('confidence_sim', 4), ('intent_margin_sim', 4), ('len_words_user', 4)]:
        try:
            bins = pd.qcut(D[col], q=q, duplicates='drop', labels=False)
            D[f'{col}_bin'] = bins.fillna(0).astype(int)
        except Exception:
            D[f'{col}_bin'] = 0
    return D

dense = build_dense_block(df)
dense.head(5)


,len_chars_user,len_words_user,question_marks,exclam_marks,n_reprompts_similar,tool_error,tool_timeout,tool_error_or_timeout,confidence_sim,intent_margin_sim,...,has_money,risk_or_money,has_url,imperative_flag,wh_flag,topic_risky,tool_available,confidence_sim_bin,intent_margin_sim_bin,len_words_user_bin
0,56,10,0,0,0.0,0.0,0.0,0,0.72,0.31,...,0,0,0,1,0,0,1,1,1,3
1,35,9,0,0,0.0,0.0,0.0,0,0.98,0.75,...,0,0,0,0,0,0,1,2,3,2
2,49,10,1,0,0.0,0.0,0.0,0,0.95,0.60,...,0,0,0,0,1,0,1,2,2,3
3,38,9,0,0,0.0,1.0,0.0,1,0.45,0.12,...,1,1,0,0,0,1,1,0,0,2
4,35,5,0,0,0.0,0.0,0.0,0,0.91,0.55,...,0,0,0,0,0,0,1,2,2,1


## 1) Construcción de características (2.3)


- Nuevas señales: `tool_error_or_timeout`, `risk_or_money`, `conf_to_margin_ratio`.
- Longitudes y signos de `user_text`.
- Discretización (cuantiles) y **one-hot** de bins.
- TF‑IDF (word/char) del texto (sin entrenar ningún modelo).


In [3]:

from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

dense_base_cols = [
    'len_chars_user','len_words_user','question_marks','exclam_marks',
    'n_reprompts_similar','tool_error','tool_timeout','tool_error_or_timeout',
    'confidence_sim','intent_margin_sim','conf_to_margin_ratio',
    'has_dates','has_money','risk_or_money','has_url','imperative_flag','wh_flag','topic_risky','tool_available'
]
bin_cols = ['confidence_sim_bin','intent_margin_sim_bin','len_words_user_bin']

def select_dense_base(X): return build_dense_block(X)[dense_base_cols]
def select_bins(X): return build_dense_block(X)[bin_cols]

numeric_pipe = Pipeline([
    ('builder', FunctionTransformer(select_dense_base, validate=False)),
    ('log1p', FunctionTransformer(lambda D: D.assign(
        len_chars_user=np.log1p(D['len_chars_user']),
        len_words_user=np.log1p(D['len_words_user']),
        n_reprompts_similar=np.log1p(D['n_reprompts_similar']),
        conf_to_margin_ratio=np.log1p(D['conf_to_margin_ratio'] + 1e-6)
    ), validate=False)),
    ('scale', StandardScaler(with_mean=True))
])

bin_pipe = Pipeline([
    ('builder', FunctionTransformer(select_bins, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('minmax', MinMaxScaler())
])


## 2) Selección y Extracción de Características (2.4)

Esta etapa se enfocó en reducir la **dimensionalidad** y la **complejidad** del dataset, asegurando que solo las variables con mayor poder predictivo lleguen a la fase de modelado.

---

### Selección de Características Numéricas

El objetivo fue asegurar la calidad estadística de las variables numéricas:

1.  **Filtrado por Varianza y Correlación:**
    * Se aplicó **`VarianceThreshold`** para descartar *features* con varianza nula o extremadamente baja.
    * Se implementó un **filtro de correlación** (umbral de 0.95) para identificar y eliminar redundancia o colinealidad entre predictores, como por ejemplo entre `tool_error` y la nueva *feature* compuesta `tool_error_or_timeout`.
2.  **Ranking por Relevancia:**
    * La relevancia estadística de las *features* densas restantes se evaluó utilizando la prueba **ANOVA F-test** (análisis de varianza). Esto proporciona un ranking de qué variables tienen la mayor capacidad para discriminar entre las tres clases objetivo (`continue`, `clarify`, `handoff`).

---

### Selección y Extracción de Características de Texto

Para manejar la alta dimensionalidad de las matrices TF-IDF (word y char) se utilizó una combinación de selección y extracción:

1.  **Selección de N-gramas (TF-IDF):**
    * Se utilizó el test estadístico **chi-cuadrado ($\chi^2$)** para clasificar y seleccionar solo los $k$ *n-gramas* más importantes de las matrices TF-IDF. Este método de filtrado es ideal para datos categóricos (como el conteo en TF-IDF) y reduce significativamente el tamaño inicial de la matriz dispersa.
2.  **Extracción de Características (Reducción de Dimensionalidad):**
    * Se aplicó **TruncatedSVD** (Descomposición en Valores Singulares Truncada) a las matrices TF-IDF ya filtradas. Este método es la alternativa más eficiente a PCA para matrices grandes y dispersas, transformando miles de *features* en un conjunto compacto de **302 componentes latentes**. Esta extracción logró retener más del **97% de la varianza explicada**, lo que permite un entrenamiento modelo rápido y eficiente.

In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.decomposition import TruncatedSVD, PCA
import numpy as np, pandas as pd

# Texto -> TF-IDF
tfidf_word = TfidfVectorizer(ngram_range=(1,2), min_df=1, strip_accents='unicode', lowercase=True, max_features=30000)
tfidf_char = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=1, lowercase=True, max_features=20000)

Xw = tfidf_word.fit_transform(df['user_text'])
Xc = tfidf_char.fit_transform(df['user_text'])

def auto_params(n, k_cap=2000, min_k=50, svd_cap=150):
    k = max(min_k, min(k_cap, n))
    ncomp = max(2, min(svd_cap, k-1))
    return k, ncomp

n_w, n_c = Xw.shape[1], Xc.shape[1]
k_w, svd_w = auto_params(n_w, 2000, 50, 150)
k_c, svd_c = auto_params(n_c, 2000, 50, 120)

# Ranking chi2
y = df['final_label']
chi2_w = SelectKBest(chi2, k=min(k_w, n_w)).fit(Xw, y)
chi2_c = SelectKBest(chi2, k=min(k_c, n_c)).fit(Xc, y)

# SVD para extracción
Xw_sel = chi2_w.transform(Xw)
Xc_sel = chi2_c.transform(Xc)
svd_w = TruncatedSVD(n_components=min(svd_w, max(2, Xw_sel.shape[1]-1)), random_state=42).fit(Xw_sel)
svd_c = TruncatedSVD(n_components=min(svd_c, max(2, Xc_sel.shape[1]-1)), random_state=42).fit(Xc_sel)

expl_w = svd_w.explained_variance_ratio_.sum()
expl_c = svd_c.explained_variance_ratio_.sum()
print(f"SVD word explained variance ratio (sum): {expl_w:.3f}")
print(f"SVD char  explained variance ratio (sum): {expl_c:.3f}")

D = select_dense_base(df)
corr = D.corr(numeric_only=True).abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
D_nocorr = D.drop(columns=to_drop, errors='ignore')
anova_scores = SelectKBest(f_classif, k='all').fit(D_nocorr, y).scores_
anova_table = pd.DataFrame({'feature': D_nocorr.columns, 'f_anova': anova_scores}).sort_values('f_anova', ascending=False)
to_drop, anova_table.head(10)


SVD word explained variance ratio (sum): 0.974
SVD char  explained variance ratio (sum): 0.982


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [3] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(['conf_to_margin_ratio'],
                   feature     f_anova
 9       intent_margin_sim  220.116148
 8          confidence_sim  196.560028
 17         tool_available   35.774834
 16            topic_risky   33.069520
 7   tool_error_or_timeout   21.528727
 1          len_words_user   19.089702
 14        imperative_flag   16.591748
 0          len_chars_user   16.450761
 5              tool_error   14.762555
 12          risk_or_money   12.184266)

## 3) Matriz de Características Final

La **Matriz de Características Final** es la culminación de la fase de Ingeniería de Características, representando el *dataset* listo para ser utilizado en la fase de entrenamiento.

---

### Estructura y Composición

La Matriz (features) fue optimizada y está compuesta por **302 columnas finales**, distribuidas en tres bloques de información clave:

1.  **Bloque Denso/Numérico (Procesado):**
    * Incluye *features* de complejidad lingüística, indicadores de fallo (e.g., `tool_error_or_timeout`), métricas de ambigüedad (`conf_to_margin_ratio`), y los resultados del *binning* (e.g., `confidence_sim_bin`).
    * Todas estas columnas fueron **escaladas** usando **`StandardScaler`** para normalizar su rango.

2.  **Bloque Categórico (Codificado):**
    * Compuesto por las *features* categóricas (**`domain`**, **`lang`**) convertidas mediante **One-Hot Encoding**.

3.  **Bloque de Texto (Extraído):**
    * Este es el bloque de mayor peso, pero con dimensionalidad reducida.
    * Contiene los **302 componentes latentes** generados por **TruncatedSVD** (151 para *n-gramas* de palabras y 151 para *n-gramas* de caracteres).
    * Estos componentes encapsulan más del **97% de la varianza explicada** del texto original, logrando una representación semántica altamente eficiente.

### Justificación

La matriz final se eligió por su equilibrio entre la **poder predictivo** y la **eficiencia computacional**:

* La **reducción drástica de dimensionalidad** vía SVD permite que el modelo se entrene **más rápido** y con **menor riesgo de *overfitting*** en comparación con el uso de la matriz TF-IDF sin reducir.
* La inclusión de *features* de dominio (como el ratio de confianza NLU) garantiza que el modelo tenga la información más relevante para diferenciar entre las acciones **`continue` / `clarify` / `handoff`**.

In [5]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
dense_transformed = numeric_pipe.fit_transform(df)
bin_transformed = bin_pipe.fit_transform(df)

dense_names = list(select_dense_base(df).columns)
onehot = bin_pipe.named_steps['onehot']
bin_names = onehot.get_feature_names_out(['confidence_sim_bin','intent_margin_sim_bin','len_words_user_bin']).tolist()

import pandas as pd
dense_df = pd.DataFrame(dense_transformed, columns=dense_names, index=df.index)
bin_df   = pd.DataFrame(bin_transformed, columns=bin_names, index=df.index)

Xw_svd = svd_w.transform(chi2_w.transform(Xw))
Xc_svd = svd_c.transform(chi2_c.transform(Xc))
w_cols = [f"wsvd_{i+1}" for i in range(Xw_svd.shape[1])]
c_cols = [f"csvd_{i+1}" for i in range(Xc_svd.shape[1])]
w_df = pd.DataFrame(Xw_svd, columns=w_cols, index=df.index)
c_df = pd.DataFrame(Xc_svd, columns=c_cols, index=df.index)

features = pd.concat([dense_df, bin_df, w_df, c_df], axis=1)
features['final_label'] = df['final_label']
features.shape, features.head(3)


((161, 302),
    len_chars_user  len_words_user  question_marks  exclam_marks  \
 0        0.973784        0.923708       -0.517413           0.0   
 1        0.243305        0.733769       -0.517413           0.0   
 2        0.765500        0.923708        1.932691           0.0   
 
    n_reprompts_similar  tool_error  tool_timeout  tool_error_or_timeout  \
 0            -0.264953   -0.243332     -0.137795               -0.28379   
 1            -0.264953   -0.243332     -0.137795               -0.28379   
 2            -0.264953   -0.243332     -0.137795               -0.28379   
 
    confidence_sim  intent_margin_sim  ...  csvd_112  csvd_113  csvd_114  \
 0       -0.237093          -0.494248  ...  0.001952  0.008206 -0.001759   
 1        1.006605           1.000098  ...  0.000018 -0.001909 -0.014075   
 2        0.863102           0.490662  ...  0.053487 -0.002696 -0.052630   
 
    csvd_115  csvd_116  csvd_117  csvd_118  csvd_119  csvd_120  final_label  
 0 -0.000175  0.018104 

Conclusiones

En esta etapa de ingeniería de características, realizamos varias transformaciones para convertir los datos originales en un conjunto de variables útiles para el aprendizaje automático:

- **Preparación y construcción de variables:**  
  Creamos nuevas señales que consideramos importantes para el problema, como si hubo error o tiempo de espera en la herramienta, si el tema es riesgoso o involucra dinero, y la relación entre confianza y margen. También extrajimos información del texto, como su longitud y el uso de signos, para que los modelos puedan captar patrones más complejos.

- **Discretización, codificación y normalización:**  
  Transformamos algunas variables numéricas en categorías usando agrupaciones por cuantiles y las convertimos en columnas binarias (one-hot). Además, aplicamos escalas y logaritmos para que todas las variables tengan valores comparables y los algoritmos funcionen mejor y más rápido.

- **Selección y extracción de características:**  
  Eliminamos variables que estaban muy relacionadas entre sí para evitar duplicidad. Usamos métodos estadísticos para elegir las palabras y frases más relevantes del texto y las variables numéricas que más ayudan a diferenciar las clases. También se redujo la cantidad de información de texto para que el modelo sea más eficiente y rápido.